In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_pickle('./preprocessed_data.pkl')
data.index

RangeIndex(start=0, stop=35728, step=1)

In [3]:
print(data.columns.tolist())

['YEAR', 'ITERATION', 'ID_HH', 'RHoMIS_ID', 'GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Altitude', 'HHsizemembers', 'HHsizeMAE', 'Head_EducationLevel', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'WorstFoodSecMonth', 'BestFoodSecMonth', 'NrofMonthsFoodInsecure', 'PPI_Threshold', 'PPI_Likelihood', 'HFIAS_status', 'FIES_Score', 'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason', 'score_HDDS_purchasedGoodSeason', 'score_HDDS_BadSeason', 'score_HDDS_farmbasedBadSeason', 'score_HDDS_purchasedBadSeason', 'TVA_USD_PPP_pmae_pday', 'currency_conversion_factor', 'total_income_USD_PPP_pHH_Yr', 'offfarm_income_USD_PPP_pHH_Yr', 'farm_income_USD_PPP_pHH_Yr', 'value_farm_produce_USD_PPP_pHH_Yr', 'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'value_crop_consumed_USD_PPP_pHH_Yr', 'livestock_prodsales_USD_PPP_pHH_Yr', 'value_livestock_production_USD_PPP_pHH_Yr', 'value_livestock_prod_consumed_USD_PPP_pHH_Yr', 'Market_Orientation', 'Livestock_Orientation', 'Food_Availability_kCal

In [4]:
data_pre = data[['GPS_LAT', 'GPS_ALT', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'Food_Availability_kCal_MAE_day', 'Food_Self_Sufficiency_kCal_MAE_day',
'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'GHGEmissions']]

In [5]:
data_pre

,GPS_LAT,GPS_ALT,LandOwned,LandCultivated,LivestockHoldings,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,crop_sales_USD_PPP_pHH_Yr,value_crop_produce_USD_PPP_pHH_Yr,GHGEmissions
0,11.19,254.00,10.0,2.0,1.04,4621.267353,754.360895,164.857702,242.318219,702.949550
1,11.19,346.00,NaN,1.2,4.00,2503.023263,2303.522017,0.000000,279.844923,94415.087935
2,11.19,324.00,2.0,0.8,0.30,381.108158,381.108158,0.000000,23.278518,113.654565
3,11.19,326.00,NaN,0.8,5.43,9948.141738,620.787518,228.969031,255.682085,43062.200002
4,11.22,308.00,10.0,4.0,2.12,2209.261281,1767.543491,22.896903,184.734710,13331.792068
...,...,...,...,...,...,...,...,...,...,...
35723,-13.82,1119.60,8.0,8.0,3.58,21491.305517,12770.090428,2970.040440,9187.374358,34884.136689
35724,-13.82,1128.85,NaN,14.0,8.80,11260.499490,4217.809731,4501.379943,8866.040295,79114.978038
35725,-13.82,1132.41,5.0,6.0,5.48,4247.172555,1221.939610,1083.897904,2007.669471,46350.365405
35726,-13.82,1136.24,4.0,4.0,0.70,5918.719620,5918.719620,0.000000,2026.134892,7452.589981


In [6]:
# Get Median for features of interest
variable_median = {}
for i in data_pre.columns.tolist():
        variable_median[i]= data_pre[i].median()
print(variable_median)

{'GPS_LAT': 7.84, 'GPS_ALT': 753.0, 'LandOwned': 1.875, 'LandCultivated': 1.6, 'LivestockHoldings': 1.23, 'Food_Availability_kCal_MAE_day': 3662.678280860062, 'Food_Self_Sufficiency_kCal_MAE_day': 913.7680701975886, 'crop_sales_USD_PPP_pHH_Yr': 118.455926874732, 'value_crop_produce_USD_PPP_pHH_Yr': 795.961266386223, 'GHGEmissions': 5993.591929142858}


In [7]:
# Display Median for features of interest
data_pre.median()

GPS_LAT                                  7.840000
GPS_ALT                                753.000000
LandOwned                                1.875000
LandCultivated                           1.600000
LivestockHoldings                        1.230000
Food_Availability_kCal_MAE_day        3662.678281
Food_Self_Sufficiency_kCal_MAE_day     913.768070
crop_sales_USD_PPP_pHH_Yr              118.455927
value_crop_produce_USD_PPP_pHH_Yr      795.961266
GHGEmissions                          5993.591929
dtype: float64

In [8]:
# Replace "nan" values with respective median of feature
data_processed = data_pre.fillna(data.median())
data_processed

,GPS_LAT,GPS_ALT,LandOwned,LandCultivated,LivestockHoldings,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,crop_sales_USD_PPP_pHH_Yr,value_crop_produce_USD_PPP_pHH_Yr,GHGEmissions
0,11.19,254.00,10.000,2.0,1.04,4621.267353,754.360895,164.857702,242.318219,702.949550
1,11.19,346.00,1.875,1.2,4.00,2503.023263,2303.522017,0.000000,279.844923,94415.087935
2,11.19,324.00,2.000,0.8,0.30,381.108158,381.108158,0.000000,23.278518,113.654565
3,11.19,326.00,1.875,0.8,5.43,9948.141738,620.787518,228.969031,255.682085,43062.200002
4,11.22,308.00,10.000,4.0,2.12,2209.261281,1767.543491,22.896903,184.734710,13331.792068
...,...,...,...,...,...,...,...,...,...,...
35723,-13.82,1119.60,8.000,8.0,3.58,21491.305517,12770.090428,2970.040440,9187.374358,34884.136689
35724,-13.82,1128.85,1.875,14.0,8.80,11260.499490,4217.809731,4501.379943,8866.040295,79114.978038
35725,-13.82,1132.41,5.000,6.0,5.48,4247.172555,1221.939610,1083.897904,2007.669471,46350.365405
35726,-13.82,1136.24,4.000,4.0,0.70,5918.719620,5918.719620,0.000000,2026.134892,7452.589981


In [9]:
# check if there's still any nan values
data_processed.isnull().values.any()

False

In [10]:
data_processed.to_pickle("./median_imputed.pkl")